# Quantum Walk

A quantum walk is a quantum analogue of the classical random walk. It leverages the principles of quantum superposition and interference to explore multiple paths simultaneously. This results in faster propagation and mixing times compared to classical random walks, making quantum walks a powerful tool for quantum computing and algorithms.

In a discrete-time quantum walk, the walker moves across the vertices of a graph in discrete steps, governed by a unitary evolution. The walker's position is described by a probability amplitude, allowing it to be in a superposition of multiple positions at once. This behavior enables quantum walks to solve certain computational problems more efficiently than their classical counterparts.

design the quantum walk operator for the case of a line with 16 nodes:

In [1]:
from classiq import *
size = 4 # number of qubits to represent 16 vertices

In [2]:
#Prepares a qubit in the ∣−⟩ state
@qfunc
def prepare_minus(x: QBit):
    X(x)
    H(x)

#Implements a conditional amplitude flip based on the input state.    
@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
    aux^=(x!=0)

@qfunc
def zero_diffuzer(x: QNum):
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)
    
    

1. Quantum State Representation:
The state of the quantum walk is typically represented as 
∣
j
,
k
⟩
, where 
j
 denotes the current vertex and 
k
 represents the coin state, which helps determine the direction of the walk.

2.Coin Operator (
C
):
The coin operator is used to create a superposition of possible directions in which the walker can move. it acts over the neighbors of the vertex and can be defined as:


$$C = \sum_j |j\rangle \langle j | \otimes \left(2|\partial_j\rangle\langle\partial_j | - I\right).$$


Here, 
∣
∂
j
⟩
 represents the uniform superposition over the neighbors of vertex 
j
, and 
I
 is the identity operator. This operator spreads the walker's state across its neighboring vertices.

3. Shift Operator (
S
):
The shift operator is a bitwise-swap operator that swaps the position 
j
 with the coin state 
k
.

4. W Iteration:
The 
W
 iteration is the combination of the coin operator and the shift operator. A single step of the quantum walk involves first applying the coin operator to introduce superposition and entanglement among neighboring vertices, and then applying the shift operator to update the walker's position based on the coin state.

In [3]:
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0]* (2**size)
    if i == 0:
        prob[i + 1] = 1.0
    elif i == (2 ** size) - 1:
        prob[i - 1] = 1.0
    else:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    print(f'State={i}, prob vec ={prob}')
    
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))



In [4]:
@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
        W_iteration(i,vertices,adjacent_vertices)

In [5]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= (((vertices+adjacent_vertices)%2) ==1)

In [6]:
@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    repeat(count= x.len,
      iteration= lambda i: SWAP(x[i],y[i]))

In [7]:
@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

In [8]:
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

    allocate(size,vertices)
    hadamard_transform(vertices)
    allocate(size,adjacent_vertices)

    W_operator(vertices,adjacent_vertices)
    S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0